In [1]:
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import metrics, preprocessing

import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import LSTM, Dense, BatchNormalization, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from pyDeepInsight import ImageTransformer, LogScaler
from keras.callbacks import EarlyStopping

import pickle, joblib

2023-12-16 01:03:36.707309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
df = pd.read_csv('FlowStatsfiles.csv', low_memory=False)

In [11]:
df.head()

,timestamp,datapath,flow_id,in_port,eth_type,eth_src,eth_dst,ip_src,tp_src,ip_dst,...,protocole,pktcount,bytecount,flowdur_sec,flowdur_nsec,pktcount_sec,pktcount_nsec,bytecount_sec,bytecount_nsec,label
0,1.702643e+09,5,172.24.16.20172.24.16.10,1,2048,06:ad:7d:c1:dc:1b,0a:b1:b5:23:aa:ad,172.24.16.2,0,172.24.16.1,...,ICMP,1,98,0,371000000,0.000000,0.000000e+00,0.000000,0.000000,0
1,1.702643e+09,5,172.24.16.30172.24.16.40,5,2048,a2:5c:46:f2:34:27,20:77:6d:b7:70:8a,172.24.16.3,0,172.24.16.4,...,ICMP,46,4508,63,769000000,0.730159,5.981795e-08,71.555556,0.000006,0
2,1.702643e+09,5,172.24.16.40172.24.16.30,3,2048,20:77:6d:b7:70:8a,a2:5c:46:f2:34:27,172.24.16.4,0,172.24.16.3,...,ICMP,46,4508,63,758000000,0.730159,6.068602e-08,71.555556,0.000006,0
3,1.702643e+09,2,172.24.16.30172.24.16.40,3,2048,a2:5c:46:f2:34:27,20:77:6d:b7:70:8a,172.24.16.3,0,172.24.16.4,...,ICMP,46,4508,63,757000000,0.730159,6.076618e-08,71.555556,0.000006,0
4,1.702643e+09,2,172.24.16.40172.24.16.30,2,2048,20:77:6d:b7:70:8a,a2:5c:46:f2:34:27,172.24.16.4,0,172.24.16.3,...,ICMP,46,4508,63,752000000,0.730159,6.117021e-08,71.555556,0.000006,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136766 entries, 0 to 136765
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   timestamp       136766 non-null  float64
 1   datapath        136766 non-null  int64  
 2   flow_id         136766 non-null  object 
 3   in_port         136766 non-null  int64  
 4   eth_type        136766 non-null  int64  
 5   eth_src         136766 non-null  object 
 6   eth_dst         136766 non-null  object 
 7   ip_src          136766 non-null  object 
 8   tp_src          136766 non-null  int64  
 9   ip_dst          136766 non-null  object 
 10  tp_dst          136766 non-null  int64  
 11  icmp_code       136766 non-null  int64  
 12  icmp_type       136766 non-null  int64  
 13  protocole       136766 non-null  object 
 14  pktcount        136766 non-null  int64  
 15  bytecount       136766 non-null  int64  
 16  flowdur_sec     136766 non-null  int64  
 17  flowdur_ns

In [13]:
df.label.unique()

array([0, 1])

In [14]:
df.dropna(inplace=True)
df.drop(['timestamp','datapath','flow_id','in_port','eth_type','eth_src', 'eth_dst', 'ip_src', 'ip_dst', 'protocole', 'icmp_code', 'icmp_type', 'flowdur_sec', 'flowdur_nsec'], axis=1, inplace=True)

In [ ]:
df.label.unique()

In [ ]:
df.protocole.unique()

In [ ]:
df = pd.get_dummies(df)
df.head()

In [ ]:
df.info()

In [ ]:
df['class'] = df['label']
del df['label']

In [7]:
scaler = preprocessing.MinMaxScaler()
df_nrm= scaler.fit_transform(df)
df_nv = pd.DataFrame(df_nrm, columns=df.columns)
df_nv.head()

,tp_src,tp_dst,pktcount,bytecount,pktcount_sec,pktcount_nsec,bytecount_sec,bytecount_nsec,label
0,0.0,0.0,0.000002,4.083276e-09,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,0.0,0.0,0.000079,1.878307e-07,0.000006,1.686728e-07,2.229147e-08,3.229505e-10,0.0
2,0.0,0.0,0.000079,1.878307e-07,0.000006,1.711205e-07,2.229147e-08,3.276371e-10,0.0
3,0.0,0.0,0.000079,1.878307e-07,0.000006,1.713466e-07,2.229147e-08,3.280699e-10,0.0
4,0.0,0.0,0.000079,1.878307e-07,0.000006,1.724859e-07,2.229147e-08,3.302512e-10,0.0


In [15]:
#X = df_nv.iloc[:,1:17].astype(float)
#y = df_nv.iloc[:,-1]
# X = df_nv.drop('label', axis=1).astype(float)
# y = df_nv['label']
X = df.drop('label', axis=1).astype(float)
y = df['label']

In [16]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# KNN

In [17]:
start_time = time.time()

# GridSearchCV pour trouver les meilleurs hyperparamètres
knnc = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [3, 5, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knnc_search = GridSearchCV(knnc, param_grid=param_grid, n_jobs=-1, cv=3, scoring='accuracy', verbose=2)
knnc_search.fit(X_train, y_train)

best_params = knnc_search.best_params_
n_neighbors = best_params['n_neighbors']
weights = best_params['weights']
metric = best_params['metric']

# Utilisation des meilleurs hyperparamètres sur l'ensemble d'entraînement
KNN = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
KNN.fit(X_train, y_train)

# Prédiction et évaluation sur l'ensemble de validation
predicted_val = KNN.predict(X_val)
accuracy_val = accuracy_score(y_val, predicted_val)
print(f"Précision sur l'ensemble de validation : {round(accuracy_val * 100, 2)}%")

# Prédiction et évaluation sur l'ensemble de test
predicted_test = KNN.predict(X_test)
accuracy_test = accuracy_score(y_test, predicted_test)
print(f"Précision sur l'ensemble de test : {round(accuracy_test * 100, 2)}%")

# Affichage des résultats finaux
print("Meilleurs hyperparamètres:", best_params)
print(classification_report(predicted_test, y_test))
print(f"{((time.time()/60) - (start_time)/60):.2f} Minute(s) --- temps pour le modèle KNN ")

    
joblib.dump(KNN, 'KNN.pkl')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Précision sur l'ensemble de validation : 99.56%
Précision sur l'ensemble de test : 99.6%
Meilleurs hyperparamètres: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     10232
           1       1.00      0.99      1.00     10283

    accuracy                           1.00     20515
   macro avg       1.00      1.00      1.00     20515
weighted avg       1.00      1.00      1.00     20515

1.18 Minute(s) --- temps pour le modèle KNN 


['KNN.pkl']

# SVM

In [ ]:
start_time = time.time()

kernels = ['linear', 'poly', 'rbf', 'sigmoid']

# Liste pour stocker les précisions pour chaque noyau
accuracy_list = []

for kernel in kernels:
    # Création du modèle SVM avec le noyau actuel
    SVM = svm.SVC(kernel=kernel)

    # Entraînement du modèle
    SVM.fit(X_train, y_train)

    # Prédiction et évaluation sur l'ensemble de validation
    predicted_svm = SVM.predict(X_val)
    accuracy_val = accuracy_score(y_val, predicted_svm)
    print(f"Précision sur l'ensemble de validation du noyau {kernel} : {round(accuracy_val * 100, 2)}%")

    #Prédiction et évaluation sur l'ensemble de test
    predicted_test = SVM.predict(X_test)
    accuracy_test = accuracy_score(y_test, predicted_test)
    accuracy_list.append(accuracy_test)
    print(f"Précision sur l'ensemble de test du noyau {kernel} : {round(accuracy_test * 100, 2)}%")
    print("########################################################################")

# Sélection du meilleur noyau en fonction de la précision maximale
best_kernel = kernels[np.argmax(accuracy_list)]

# Entraînement final avec le meilleur noyau
best_SVM = svm.SVC(kernel=best_kernel)
best_SVM.fit(X_train, y_train)

# Prédiction avec le modèle optimal
predicted_svm = best_SVM.predict(X_test)

# Calcul de la précision
accuracy_svm = accuracy_score(y_test, predicted_svm)

# Affichage des résultats finaux
print(f"Précision du modèle SVM avec le meilleur noyau (noyau {best_kernel}): {round(accuracy_svm * 100, 2)}%")
print("########################################################################")
print('Le meilleur noyau est:', best_kernel)
print("########################################################################")
print(classification_report(y_test, predicted_svm))
print("########################################################################")
print("--- %s seconds --- temps pour le model SVM" % (time.time() - start_time))

# Sauvegarde du modèle avec pickle
model_serialized = pickle.dumps(best_SVM)
with open('modele_svm.pickle', 'wb') as model_file:
    model_file.write(model_serialized)

joblib.dump(best_SVM, 'best_SVM.pkl')

# RF

In [ ]:
start_time = time.time()

# Définissez la grille d'hyperparamètres à explorer
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'n_estimators': [100, 200, 300],
    'max_depth': [None],
    'min_samples_split': [2],
    'max_features': ['sqrt']
}

# Créez le modèle RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Utilisez GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Obtenir les meilleurs hyperparamètres trouvés
rf_classifier_best = grid_search.best_estimator_

# Entraînez un modèle RandomForest avec les meilleurs hyperparamètres
rf_classifier_best.fit(X_train, y_train)

# Prédiction et évaluation sur l'ensemble de validation
predicted_val = rf_classifier_best.predict(X_val)
accuracy_val = accuracy_score(y_val, predicted_val)
print(f"Précision sur l'ensemble de validation : {round(accuracy_val * 100, 2)}%")

# Prédisez les étiquettes
predicted_rf = rf_classifier_best.predict(X_test)
accuracy_rf = accuracy_score(y_test, predicted_rf)

# Affichage des résultats
print(f"Random Forest Précision: {round(accuracy_rf * 100, 2)}%")
print(classification_report(predicted_rf, y_test))
print("--- %s secondes --- temps pour le Random Forest " % (time.time() - start_time))

# Sauvegarde du modèle avec pickle
model_serialized = pickle.dumps(rf_classifier_best)
with open('modele_rf.pickle', 'wb') as model_file:
    model_file.write(model_serialized)
    
joblib.dump(rf_classifier_best, 'rf_classifier_best.pkl')

# LR

In [20]:
# Paramètres du modèle
C_value = 0.03
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

results_lr = []

start_time = time.time()
best_accuracy = 0
best_solver = None

for solver in solvers:
    # Entraînement du modèle
    logreg_model = LogisticRegression(C=C_value, solver=solver, max_iter=1000, tol=1e-3).fit(X_train, y_train)
    predicted_lr = logreg_model.predict(X_test)
    accuracy_lr = accuracy_score(y_test, predicted_lr)
    
    results_lr.append({
        'solver': solver,
        'accuracy': f'{accuracy_lr * 100:.2f}%',
        'coefficients': {'W': logreg_model.coef_, 'b': logreg_model.intercept_}
    })

    if accuracy_lr > best_accuracy:
        best_accuracy = accuracy_lr
        best_solver = solver

# Entraînement final avec le meilleur solveur
logreg_model = LogisticRegression(C=C_value, solver=best_solver).fit(X_train, y_train)


# Prédiction et évaluation sur l'ensemble de validation
predicted_val = logreg_model.predict(X_val)
accuracy_val = accuracy_score(y_val, predicted_val)
print(f"Précision sur l'ensemble de validation : {round(accuracy_val * 100, 2)}%")

# Prédisez les étiquettes

predicted_lr = logreg_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, predicted_lr)

# Affichage des résultats
print(f"Précision: {accuracy_lr * 100:.2f}%\n")
print("########################################################################")
print('Meilleur solveur :', best_solver)
print("########################################################################")
print(classification_report(predicted_lr, y_test), '\n')
print("########################################################################")
print(f"{((time.time()/60) - (start_time)/60):.2f} Minutes --- temps pour la régression logistique ")
#print(f"--- {(time.time() - start_time:.2f)} Minutes --- temps pour la régression logistique")

# Sauvegarde du modèle avec pickle
model_serialized = pickle.dumps(logreg_model)
with open('modele_lr.pickle', 'wb') as model_file:
    model_file.write(model_serialized)
    
joblib.dump(logreg_model, 'logreg_model.pkl')

/home/aizen/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:457: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/aizen/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/aizen/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


Précision sur l'ensemble de validation : 99.28%
Précision: 99.29%

########################################################################
Meilleur solveur : lbfgs
########################################################################
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     10251
           1       1.00      0.99      0.99     10264

    accuracy                           0.99     20515
   macro avg       0.99      0.99      0.99     20515
weighted avg       0.99      0.99      0.99     20515
 

########################################################################
28.74 Minutes --- temps pour la régression logistique 


/home/aizen/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['logreg_model.pkl']

# DT

In [19]:
start_time = time.time()
dt_classifier = DecisionTreeClassifier()

# Recherche des hyperparamètres optimaux
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 2, 3, 4, 5],
    'max_leaf_nodes': [2, 3, 4, 5]
}

dt_search = GridSearchCV(dt_classifier, param_grid=param_grid, n_jobs=-1, cv=5, scoring='accuracy', verbose=2)
dt_search.fit(X_train, y_train)

criterion = dt_search.best_params_['criterion']
max_depth = dt_search.best_params_['max_depth']
max_leaf_nodes = dt_search.best_params_['max_leaf_nodes']

# Entraînement du modèle avec les hyperparamètres optimaux
dt_classifier = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, max_leaf_nodes=max_leaf_nodes)
dt_classifier.fit(X_train, y_train)

# Prédiction et évaluation sur l'ensemble de validation
predicted_val = dt_classifier.predict(X_val)
accuracy_val = accuracy_score(y_val, predicted_val)
print(f"Précision sur l'ensemble de validation : {round(accuracy_val * 100, 2)}%")


# Prédiction et évaluation du modèle
predicted_dt = dt_classifier.predict(X_test)
accuracy_dt = accuracy_score(y_test, predicted_dt)

# Stockage des résultats
results = {
    'criterion': criterion,
    'max_depth': max_depth,
    'max_leaf_nodes': max_leaf_nodes,
    'accuracy': round(accuracy_dt * 100, 2)
}

# Affichage des résultats
print(f"criterion: {criterion}, max depth: {max_depth}, max_leaf: {max_leaf_nodes}")
print(f"La précision est : {results['accuracy']}%")
print("########################################################################")
print(classification_report(predicted_dt, y_test))
print("########################################################################")

print(f"{((time.time()/60) - (start_time)/60):.2f} Minutes secondes --- temps pour l'arbre de décision ")

# Sauvegarde du modèle avec pickle
model_serialized = pickle.dumps(dt_classifier)
with open('modele_dt.pickle', 'wb') as model_file:
    model_file.write(model_serialized)
    
joblib.dump(dt_classifier, 'dt_classifier.pkl')

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Précision sur l'ensemble de validation : 99.43%
criterion: gini, max depth: None, max_leaf: 5
La précision est : 99.5%
########################################################################
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     10251
           1       1.00      0.99      1.00     10264

    accuracy                           1.00     20515
   macro avg       1.00      1.00      1.00     20515
weighted avg       1.00      1.00      1.00     20515

########################################################################
0.26 Minutes secondes --- temps pour l'arbre de décision 


['dt_classifier.pkl']

[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=2; total time=   0.1s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=3; total time=   0.1s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=3; total time=   0.1s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=3; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=4; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=5; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=5; total time=   0.2s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=2; total time=   0.2s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=3; total time=   0.2s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=4; total time=   0.2s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=5; total time=   0.3s
[CV] END ......criterion=gini, max_depth=3, max_leaf_nodes=2; total time=   0.3s
[CV] END ......criterion=gin

[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=2; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=2; total time=   0.3s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=3; total time=   0.3s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=4; total time=   0.3s
[CV] END ...criterion=gini, max_depth=None, max_leaf_nodes=5; total time=   0.3s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=2; total time=   0.2s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=3; total time=   0.2s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=4; total time=   0.1s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=5; total time=   0.1s
[CV] END ......criterion=gini, max_depth=2, max_leaf_nodes=5; total time=   0.1s
[CV] END ......criterion=gini, max_depth=3, max_leaf_nodes=2; total time=   0.1s
[CV] END ......criterion=gini, max_depth=3, max_leaf_nodes=3; total time=   0.3s
[CV] END ......criterion=gin

In [ ]:
# Création du modèle CNN 1D
model = models.Sequential()

# Ajout d'une couche de convolution 1D avec activation ReLU et pooling
model.add(layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(15, 1)))
model.add(layers.MaxPooling1D(pool_size=2))

# Ajout d'une deuxième couche de convolution 1D avec activation ReLU et pooling
model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))

# Aplatir les données pour les passer à une couche dense (fully connected)
model.add(layers.Flatten())

# Ajout d'une couche fully connected avec activation ReLU
model.add(layers.Dense(128, activation='relu'))

# Ajout de la couche de sortie
model.add(layers.Dense(1, activation='sigmoid'))  # Utilisez 'softmax' si vous avez plus de deux classes

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Utilisez 'categorical_crossentropy' si vous avez plus de deux classes
              metrics=['accuracy'])

# Afficher un résumé du modèle
model.summary()


In [ ]:
# Un rappel qui arrête la formation lorsqu'il n'y aura pas d'amélioration de la perte de validation pendant 5 époques consécutives.
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True)
              
# Entraînement du modèle avec ensemble de validation
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[es])

# Évaluation du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Sauvegarde du modèle avec pickle
model_serialized = pickle.dumps(history)
with open('modele_cnn.pickle', 'wb') as model_file:
    model_file.write(model_serialized)

In [ ]:
# Créez un modèle séquentiel
model = models.Sequential()

# Couche d'entrée
model.add(Dense(128, input_shape=(12,), activation='relu'))

# Couches cachées profondes
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Couche de sortie
model.add(Dense(1, activation='sigmoid'))

# Compilez le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()

In [ ]:
# Ce rappel arrêtera la formation lorsqu'il n'y aura pas d'amélioration de la perte de validation pendant 5 époques consécutives.  
es = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=5,
    restore_best_weights=True
)

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='Adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, 
          y_train, 
          epochs=30,
          batch_size=255, 
          verbose=1, 
          validation_data=(X_val, y_val),
          callbacks=[es])
joblib.dump(model, 'model_cnn.pkl')

In [ ]:
history_dict = model.history.history

In [ ]:
import matplotlib.pyplot as plt
# loss
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1) 

# plot
plt.figure(figsize=(9, 4))
plt.plot(epochs, loss_values, 'blue',label="Perte d'entraînement")
plt.plot(epochs, val_loss_values, 'green', label='Perte de validation')
plt.title("L’exactitude et la perte de la classification binaire")
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()
plt.savefig('perte.png')

In [ ]:
# accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(9, 4))
plt.plot(epochs, acc, 'blue',label="Précision de l'entraînement")
# orange is for "orange"
plt.plot(epochs, val_acc, 'green', label='Précision des validations')
plt.title('Précision de la formation et de la validation')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.legend()
plt.savefig('precision.png')

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=250)
print("Test set accuracy = {} %".format( results[1]*100))


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
preds = np.round(model.predict(X_test),0)

print(confusion_matrix(y_test, preds)) 

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt


# Création du modèle K-means avec un nombre spécifié de clusters (k)
k = 3  # Remplacez cela par le nombre de clusters que vous souhaitez
kmeans = KMeans(n_clusters=k, random_state=42)

# Entraînement du modèle
kmeans.fit(X)

# Prédictions des clusters pour chaque observation
labels = kmeans.labels_

# Coordonnées des centroïdes
centroids = kmeans.cluster_centers_

# Ajout des labels au DataFrame d'origine (si nécessaire)
# Remplacez "df" par le nom de votre DataFrame contenant les données
df_nv['label'] = labels

# Visualisation des clusters
# Remplacez "df" par le nom de votre DataFrame contenant les données
plt.scatter(df_nv['protocol_ICMP'], df_nv['protocol_TCP'], df_nv['protocol_UDP'],c=labels, cmap='viridis', marker='o', edgecolors='black')
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, label='Centroids')
plt.title('Clustering avec K-means')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()


In [ ]:
# Créer un DataFrame avec les données
data = {
    'Algorithmes': ['Forêt Aléatoire', 'SVM', 'Régression Logistique', 'Arbre de Decision', 'KNN'],
    'Accuracy': [99.94, 95.14, 95, 95.68, 97],
    'False Alarm Rate': [None, None, 95, None, None],
    'Precision': [98.22, 97.73, 93.60, 97.95, 97.99],
    'Recall': [None, None, 95, None, 97],
    'F1-score': [None, None, 95, None, 97]
}

df = pd.DataFrame(data)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
df.set_index('Algorithmes').plot(kind='bar', ax=ax, stacked=True)

# Ajouter des légendes et des étiquettes
plt.title('Performance des Algorithmes')
plt.ylabel('Score (%)')
plt.xticks(rotation=0, ha='right')

# Afficher le graphique
plt.show()


In [ ]:
# Créer un DataFrame avec les données
data = {
    'Algorithmes': ['Forêt Aléatoire', 'SVM', 'Régression Logistique', 'Arbre de Decision', 'KNN'],
    'Precision': [98.22, 97.73, 93.60, 97.95, 97.99]
}

df = pd.DataFrame(data)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
df.set_index('Algorithmes').plot(kind='bar', ax=ax, stacked=True)

# Ajouter des légendes et des étiquettes
plt.title('Performance des Algorithmes')
plt.ylabel('Score (%)')
plt.xticks(rotation=0, ha='right')

# Afficher le graphique
plt.show()


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Charger le jeu de données

#df = pd.DataFrame(data)

# Sélectionner les colonnes pertinentes pour le clustering
X = df[['pktcount', 'bytecount', 'flowdur_sec', 'flowdur_nsec', 'pktcount_sec', 'pktcount_nsec', 'bytecount_sec', 'bytecount_nsec']]

# Définir le nombre de clusters (k)
k = 10

# Créer l'instance de l'algorithme K-moyennes
kmeans = KMeans(n_clusters=k)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled)
# Entraîner l'algorithme sur les données
kmeans.fit(X_scaled)

# Ajouter les étiquettes de cluster aux données d'origine
df['Cluster'] = kmeans.labels_

# Afficher les résultats
print(df)

# Pour visualiser, vous pouvez choisir deux caractéristiques à la fois et les afficher sur un graphique 2D.
feature1 = 'pktcount'
feature2 = 'bytecount'

plt.scatter(df[feature1], df[feature2], c=df['Cluster'], cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, X.columns.get_loc(feature1)], kmeans.cluster_centers_[:, X.columns.get_loc(feature2)], marker='X', s=200, c='red')
plt.xlabel(feature1)
plt.ylabel(feature2)
plt.title('K-moyennes Clustering')
plt.show()


In [ ]:
# Afficher quelques exemples d'étiquettes réelles et de clusters attribués
print("True Labels:", true_labels[:10])
print("Cluster Labels:", cluster_labels[:10])

# Afficher les tailles des deux tableaux
print("Size of True Labels:", len(true_labels))
print("Size of Cluster Labels:", len(cluster_labels))


In [ ]:
# Les véritables étiquettes de classe (si disponibles)
true_labels = np.array([0, 1])

# Les étiquettes de cluster attribuées par K-moyennes
cluster_labels = kmeans.labels_

# Calculer les métriques
accuracy = accuracy_score(true_labels, cluster_labels)
precision = precision_score(true_labels, cluster_labels, average='weighted')
recall = recall_score(true_labels, cluster_labels, average='weighted')
f1 = f1_score(true_labels, cluster_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Charger le dataset
# Assurez-vous que le chemin du fichier est correct
data = pd.read_csv("FlowStatsfile.csv")

# Sélectionner les colonnes pertinentes pour le clustering
selected_columns = ['pktcount', 'bytecount', 'flowdur_sec', 'flowdur_nsec', 'pktcount_sec', 'pktcount_nsec', 'bytecount_sec', 'bytecount_nsec']

# Créer un DataFrame avec les colonnes sélectionnées
X = data[selected_columns]

# Traiter les colonnes catégorielles (comme les adresses IP et le protocole)
#X = pd.get_dummies(X, columns=['ip_src', 'ip_dst', 'eth_type', 'protocole'])

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled)

# Déterminer le nombre optimal de clusters (k) en utilisant la méthode du coude (Elbow Method)
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Tracer le coude
plt.plot(range(1, 11), wcss)
plt.title('Méthode du coude')
plt.xlabel('Nombre de clusters')
plt.ylabel('WCSS')  # Within cluster sum of squares
plt.show()

# À partir du graphique du coude, choisissez le nombre optimal de clusters (k)

# Appliquer K-means avec le nombre optimal de clusters
k = 10  # Remplacez par le nombre optimal de clusters que vous avez choisi
kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
clusters = kmeans.fit_predict(X_scaled)

# Ajouter les résultats du clustering au DataFrame original
data['cluster'] = clusters
data['Cluster'] = kmeans.labels_

# Afficher les résultats
print(data[['pktcount', 'bytecount', 'flowdur_sec', 'flowdur_nsec', 'pktcount_sec', 'pktcount_nsec', 'bytecount_sec', 'bytecount_nsec']])


In [ ]:
# Pour visualiser, vous pouvez choisir deux caractéristiques à la fois et les afficher sur un graphique 2D.
feature1 = 'pktcount'
feature2 = 'bytecount'

plt.scatter(data[feature1], data[feature2], c=clusters, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, X.columns.get_loc(feature1)], kmeans.cluster_centers_[:, X.columns.get_loc(feature2)], marker='X', s=200, c='red')
plt.xlabel(feature1)
plt.ylabel(feature2)
plt.title('K-moyennes Clustering')
plt.show()

In [ ]:
# Afficher quelques exemples d'étiquettes réelles et de clusters attribués
print("True Labels:", true_labels[:10])
print("Cluster Labels:", cluster_labels[:10])

# Afficher les tailles des deux tableaux
print("Size of True Labels:", len(true_labels))
print("Size of Cluster Labels:", len(cluster_labels))
